### Get the top 10 records in the trip table from the first month

In [1]:
%%sql

select * from nyctaxidb.trip where month=1 limit 10

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1595680205259_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

### Get the top 10 records in the fare table from the first month:

In [2]:
%%sql 

select * from nyctaxidb.fare where month=1 limit 10

### View the number of records in each of the 12 partitions

In [3]:
%%sql

select month, count(*) from nyctaxidb.trip group by month

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 10



### Total # of records

In [4]:
%%sql

select count(*) from nyctaxidb.fare

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 12



### Trip distribution by medallion

In [5]:
%%sql

SELECT medallion, COUNT(*) as med_count
FROM nyctaxidb.fare
WHERE month<=3
GROUP BY medallion
HAVING med_count > 100
ORDER BY med_count desc

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 21



### Trip distribution by medallion and hack license

In [6]:
%%sql

SELECT medallion, hack_license, COUNT(*) as trip_count
FROM nyctaxidb.fare
WHERE month=1
GROUP BY medallion, hack_license
HAVING trip_count > 100
ORDER BY trip_count desc

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 24



### Assessing data quality by checking for invalid longitude or latitude records

In [7]:
%%sql

SELECT COUNT(*) FROM nyctaxidb.trip
    WHERE month=1
    AND  (CAST(pickup_longitude AS float) NOT BETWEEN -90 AND -30
    OR    CAST(pickup_latitude AS float) NOT BETWEEN 30 AND 90
    OR    CAST(dropoff_longitude AS float) NOT BETWEEN -90 AND -30
    OR    CAST(dropoff_latitude AS float) NOT BETWEEN 30 AND 90)

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 27



### Binary class distributions of trip tips

In [8]:
%%sql

SELECT tipped, COUNT(*) AS tip_freq
FROM
(
    SELECT if(tip_amount > 0, 1, 0) as tipped, tip_amount
    FROM nyctaxidb.fare
)tc
GROUP BY tipped

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 32



### Class distributions in the multiclass setting

In [9]:
%%sql

SELECT tip_class, COUNT(*) AS tip_freq
FROM
(
    SELECT if(tip_amount=0, 0,
        if(tip_amount>0 and tip_amount<=5, 1,
        if(tip_amount>5 and tip_amount<=10, 2,
        if(tip_amount>10 and tip_amount<=20, 3, 4)))) as tip_class, tip_amount
    FROM nyctaxidb.fare
)tc
GROUP BY tip_class

Exception in thread Thread-22:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 35



### Compute the direct distance between two longitude-latitude locations

In [10]:
%%sql

select pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, trip_distance, trip_time_in_secs,
3959*2*2*atan((1-sqrt(1-pow(sin((dropoff_latitude-pickup_latitude)
 *radians(180)/180/2),2)-cos(pickup_latitude*radians(180)/180)
 *cos(dropoff_latitude*radians(180)/180)*pow(sin((dropoff_longitude-pickup_longitude)*radians(180)/180/2),2)))
 /sqrt(pow(sin((dropoff_latitude-pickup_latitude)*radians(180)/180/2),2)
 +cos(pickup_latitude*radians(180)/180)*cos(dropoff_latitude*radians(180)/180)*
 pow(sin((dropoff_longitude-pickup_longitude)*radians(180)/180/2),2))) as direct_distance
from nyctaxidb.trip
where month=1
and pickup_longitude between -90 and -30
and pickup_latitude between 30 and 90
and dropoff_longitude between -90 and -30
and dropoff_latitude between 30 and 90

Exception in thread Thread-23:
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/bin/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 196, in _check_jobs
    self._send_msgs_for_fast_job(next_job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 290, in _send_msgs_for_fast_job
    self._send_job_start(job)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/sparkprogressindicator/sparkmonitorbackend.py", line 375, in _send_job_start
    stage = stage_dict[stageId]
KeyError: 38

